# Búsqueda semántica
Autor: Eric S. Tellez <eric.tellez@infotec.mx>


# Language models
En los word embeddings, cada palabra del vocabulario tiene asignado un vector denso de manera estática, fruto de su semántica basada en la hipótesis distribucional. Los modelos de lenguaje van más allá, intentando no solo tener en cuenta una palabra para el vector, si no que el vector mismo es dependiente del contexto, por lo cual puede desambiguar de manera natural palabras idénticas (homónimos) usando dicha información contextual. Adicionalmente, capturan información relevante de grandes corpus de texto, aportando muchas veces información de un mundo (i.e., si se entrenan usando la wikipedia, tendrán información relavante de multiples dominios).

La punta de lanza de los modelos de lenguaje es el aprendizaje profundo, y más precisamente, con el uso de _transformers_. Uno de los modelos de lenguaje más utilizado es BERT [@VSPU2017]. La persona interesada más sobre ellos <https://huggingface.co/>. 
Cuando la información esta poco específicada, las palabras adecuadas podrían ser dificiles de tener o limitar. En estos casos, la representaciones semánticas que permiten buscar _lo que se desea_ por medio de conceptos nos acerca más a la posibilidad de obtener información útil.

En lo que resta de este notebook se estarán viendo como usar los modelos, y más precisamente los embeddings para realizar tareas de recuperación de información.


In [1]:
using Pkg
Pkg.activate(".")

  Activating project at `~/Cursos/IR-2024/Unidades`


In [2]:
using SimilaritySearch, LinearAlgebra, HypertextLiteral, CodecZlib, JSON, Random, StatsBase, DataFrames, JLD2
using Downloads: download

# Using Transformers as encoders
## Using SentenceBert

In [3]:
using PyCall # note that I already installed sentence-tranformers package and configured PyCall
pushfirst!(pyimport("sys")."path", "") # this is necessary to load modules in the current path
sbert = pyimport("sbertencoder")

PyObject <module 'sbertencoder' from '/home/sadit/Cursos/IR-2024/Unidades/sbertencoder.py'>

In [12]:
langmodel = sbert.TextEncoder()

PyObject <sbertencoder.TextEncoder object at 0x7ff79b467150>

In [30]:
function encode_text(model, text::AbstractString; normalize=true)
    e = model.encode(text)
    normalize && normalize!(e)
    e
end

function encode_corpus(model, corpus::AbstractVector; normalize=true)
    n = length(corpus)
    e = model.encode(corpus)
    E = permutedims(e)
    @assert size(E, 1) == 384 # only for sbert
    if normalize
        for c in eachcol(E)
            normalize!(c)
        end
    end

    E
end

function create_index(db, recall)
    dist = NormalizedCosineDistance()
    index = SearchGraph(; dist, db, verbose=false)
    index!(index; callbacks=SearchGraphCallbacks(MinRecall(recall)))
    optimize!(index, MinRecall(recall))
    index
end

create_index (generic function with 1 method)

In [9]:
display(@htl "<h1>Cargando el corpus</h1>")

include("read_datasets.jl")
train, test = DataFrame.(read_news())
first(train, 3)

<h1>Cargando el corpus</h1>

Row,text,labels
,String,String
1,RT @SanGonAzc: @RadioIndeMexico @FiscaliaNL @FGRMexico @lopezobrador_ @AldoFasciZ @RicardoMeb se van a seguir haciendo tontos?? #debaniesc…,AdriDelgadoRuiz
2,"“El periodismo serio y responsable es una herramienta esencial para salvaguardar y fortalecer el sistema democrático en México y el mundo, y hoy se debe fortalecer esa profesión”, afirmó el periodista y escritor, Harris Whitbeck @harriswhitbeck\n#ElDedoEnLaLLaga #HeraldoRadio",AdriDelgadoRuiz
3,"""...Sí esperamos que se aumente la tasa en 25 puntos base en las siguientes dos reuniones y continuar así durante el primer trimestre del 2022”: @raguilar_abe",AdriDelgadoRuiz


In [33]:
function load_or_compute_embeddings(langmodel, corpus, h5filename; normalize=false)
    if !isfile(h5filename)
        @time X = encode_corpus(langmodel, test.text)
        jldsave(h5filename; emb=X)
    else
        X = jldopen(h5filename) do f
            f["emb"]
        end
    end

    if normalize
        for c in eachcol(X)
            normalize!(c)
        end
    end

    X
end

load_or_compute_embeddings (generic function with 1 method)

In [34]:
trainfile = "data/spanish-twitter-news-and-opinions-top25-68.train--sbert-multi-L12-v2.h5"
testfile = "data/spanish-twitter-news-and-opinions-top25-68.test--sbert-multi-L12-v2.h5"

Xtrain = load_or_compute_embeddings(langmodel, train.text, trainfile)
Xtest = load_or_compute_embeddings(langmodel, test.text, testfile)

size(Xtrain), size(Xtest)

((384, 30244), (384, 3361))

Matrix{Float32} (alias for Array{Float32, 2})

### Se crea el índice métrico

In [35]:
@time index = create_index(MatrixDatabase(Xtrain), 0.95)

  0.649584 seconds (602.12 k allocations: 55.760 MiB, 4.58% compilation time)


SearchGraph{NormalizedCosineDistance, MatrixDatabase{Matrix{Float32}}, SimilaritySearch.AdjacencyLists.AdjacencyList{UInt32}, BeamSearch}
  dist: NormalizedCosineDistance NormalizedCosineDistance()
  db: MatrixDatabase{Matrix{Float32}}
  adj: SimilaritySearch.AdjacencyLists.AdjacencyList{UInt32}
  hints: Array{Int32}((105,)) Int32[60, 165, 177, 178, 222, 386, 478, 537, 566, 567  …  2443, 2475, 2482, 2495, 2515, 2546, 2577, 2608, 2616, 2653]
  search_algo: BeamSearch
  len: Base.RefValue{Int64}
  verbose: Bool false


### Búsqueda de todos los vecinos cercanos de test en train, observe la conveniencia del uso de un índice

Entre más sean las consultas más se ve la bondad

In [36]:
let k = 32
    @time Iknns, Idists = searchbatch(index, MatrixDatabase(Xtest), k)
    exs = ExhaustiveSearch(; dist=distance(index), db=database(index))
    @time Eknns, Edists = searchbatch(exs, MatrixDatabase(Xtest), k)
    @show macrorecall(Eknns, Iknns)
end

  0.059625 seconds (6.78 k allocations: 1.195 MiB)
  0.416515 seconds (133 allocations: 851.484 KiB)
macrorecall(Eknns, Iknns) = 0.9418327878607557


0.9418327878607557

### Búsqueda y presentación de los resultados

In [37]:
#Eindex = ExhaustiveSearch(; dist=distance(index), db=database(index))
E = (; langmodel, index, data=train); # para simplificar algunas funciones

In [38]:
function search_and_display(E, qtext, k)
    res = KnnResult(k)
    q = encode_text(E.langmodel, qtext)

    @time search(index, q, res)
    
    L = []
    for (j, p) in enumerate(res)
        id, d = p.id, p.weight
        text, label = E.data.text[id], E.data.labels[id]
        push!(L, @htl "<tr><td>$j</td><td>$(id)</td><td>$(round(d, digits=3))</td> <td>$(label)</td><td> $(text)</td> </tr>")
    end

    display(@htl """<h2>resultados for "$qtext"</h2>
    <table>
    <th>  <td>id</td> <td>dist</td> <td>user</td> </td>message<td> </th>
        $L
    </table>
    """)
end


search_and_display (generic function with 1 method)

In [47]:
display(@htl "<h1>Ejemplos de búsqueda</h1>")
#search_and_display(E, E.data.corpus[1], 7)
search_and_display(E, "busqueda de personas desaparecidas", 7)
search_and_display(E, "sobre el covid 19 y el coronavirus", 7)
search_and_display(E, "arte y cine", 7)
search_and_display(E, "sobre el narcotráfico", 7)

<h1>Ejemplos de búsqueda</h1>

  0.000203 seconds (4 allocations: 96 bytes)


1,9927,0.284,UniNoticias,🎥 Esta familia pide ayuda para encontrar a una mujer de 60 años reportada como desaparecida. https://t.co/XkV0npQ9BS
2,12201,0.3,Milenio,"▶️ Continúa búsqueda de Debahni Escobar, joven de 18 años desaparecida desde el 8 de abril en Nuevo León En la búsqueda participaron cerca de 50 personas entre familiares, amigos, colectivos y sociedad civil 📺 #Milenio19h con @pedrogamboamr https://t.co/vHAaTuznT6"
3,14255,0.327,julioastillero,🚢 #EnVivo en #AstilleroInforma El tema de desaparición forzada se ha convertido en un dolor para la sociedad. Todas las personas tienen el derecho a ser buscadas y encontradas. Es el momento de echar a andar esto a rango constitucional: @Omarel44 https://t.co/qe3FatOwmv https://t.co/wuOVVFovff
4,24109,0.343,abc_es,Vidas sin rastro: la angustia de miles de búsquedas https://t.co/1YNePO0dTJ ✍ Un reportaje de Beatriz L. Echazarreta
5,24560,0.355,epigmenioibarra,"RT @jenarovillamil: Ay, diputada @Mzavalagc, qué tal si revisamos las listas de desaparecidos en la lista del sexenio calderonista"
6,1281,0.373,Milenio,"▶️ ""Las desapariciones no han frenado y nos habla de un problema presente y una deuda con las víctimas"": Sofía de Robina, abogada del Área Internacional del Centro PRODH México llegó a las 100 mil personas desaparecidas 📺 La entrevista con @elisaalanis #ElisaEnMILENIO https://t.co/3FRfarsXyh"
7,11112,0.376,julioastillero,"Desaparecida, en #Apodaca #NuevoLeón :"


  0.000144 seconds (4 allocations: 96 bytes)


1,26658,0.211,abc_es,Adiós a la mascarilla en exteriores: así quedan las restricciones por el coronavirus por comunidades https://t.co/w9sIL69Xxy
2,21235,0.213,abc_es,Qué es la variante Delta del coronavirus https://t.co/oTKQRropai
3,29257,0.216,CNNEE,Así es como surgen las variantes del nuevo coronavirus. https://t.co/qH1ggXs3KH
4,20772,0.222,abc_es,Así quedan las restricciones por el coronavirus por comunidades https://t.co/1AC0unePne
5,17655,0.23,RicardoAlemanMx,"El subsecretario de Salud, López-Gatell, confirmó el primer caso de coronavirus covid-19 con ómicron en México https://t.co/iTaO3k7DfQ"
6,4076,0.233,Radio_Formula,La altamente transmisible variante ómicron ha provocado un alza de casos de coronavirus en todo el mundo. https://t.co/N8LHD2DGFO
7,8031,0.233,PublimetroCol,Este viernes solo hay 20.221 casos activos de coronavirus en el país https://t.co/jBj5pD28Vn


  0.000162 seconds (4 allocations: 96 bytes)


1,6010,0.366,el_pais,"RT @elpais_cultura: El cineasta estrena en Lisboa ‘Andy’, su primera obra de teatro, dedicada al visionario artista del ‘pop-art’ con el qu…"
2,2188,0.391,el_pais,"""El desprecio contra la gente del cine es algo histórico, somos el oficio de 'los rojos'. Es muy grave que desde instituciones que manejan presupuestos públicos pueda despreciarse un campo de expresión artística"", lamenta Alberto San Juan https://t.co/pKVnkddEQj"
3,30110,0.415,El_Universal_Mx,RT @Univ_Cultura: ¿Cómo es que terminaste teniendo “perrhijos” y “gathijos”? Esta exposición te lo explica a través de la historia del arte…
4,21570,0.417,PublimetroCol,"Hablamos en exclusiva con dos de las mentes creativas detrás de esta película que promete enamorar a muchos al son de música, tradición, culturas y diversión. https://t.co/G0sT9XEYuK"
5,18242,0.418,EFEnoticias,"ENTREVISTA | Pedro Halffter: La convivencia con el arte debe ser en vivo, no en la pantalla. Por Sergio Jiménez Foronda https://t.co/8mZg1cEkhl https://t.co/aHH0z4Y612"
6,11035,0.419,PublimetroMX,"#Entrevista | @tokischa_ :“El arte en general es libre para expresar lo que uno sienta y se imagine"". https://t.co/MxwFvkmZQd"
7,1597,0.422,teleSURtv,"#ENVIDEO | El #ArteContemporáneo tiene diferentes representaciones, entre ellas el #LandArt Las piezas de arte de esta corriente se las realiza en el entorno natural Conoce más sobre esta puesta en escena en el siguiente vídeo https://t.co/alKzXTItPM"


  0.000195 seconds (4 allocations: 96 bytes)


1,15440,0.289,abc_es,"Drogas https://t.co/WVZaCfEuUP ✍ 'Tveo', por @rosabelmonte"
2,26384,0.336,PublimetroChile,"Tras los buenos resultados del Estudio Nacional de Drogas en Población General, publicados el...👇 https://t.co/I4HzdWR9Uj"
3,28700,0.337,RicardoAlemanMx,RT @ContrapesoC: Este es el pacto del narco y Cuauhtémoc Blanco. ¿Alguna duda del narcoestado? https://t.co/qDpAwTKcHP
4,3930,0.339,AristeguiOnline,La mayoría de los cárteles se ha centrado en el narcotráfico... hasta que en 2006 comenzaron a involucrarse en otras empresas criminales. https://t.co/LEj1AA0Dky
5,22011,0.341,AristeguiOnline,El comercio ilegal de drogas se digitaliza con la pandemia https://t.co/dzZtvcjBAA https://t.co/hrG2x5a5xs
6,26428,0.344,RicardoAlemanMx,RT @RicardoAlemanMx: “Daños colaterales”: Patricia Armendáriz sobre “pérdidas” por desabasto de medicamentos https://t.co/CyzKK301z3
7,23204,0.347,bbcmundo,"""Los recolectores de cocaína"" de Rotterdam: la vida en los sombras de los jóvenes que descargan la droga que viaja de América Latina a Europa https://t.co/1NJXa7kpgs"


In [44]:
display(@htl "<h1>Ejemplos de búsqueda (mensajes aleatorios sobre el train)</h1>")

for i in 1:3
    for qid in rand(1:length(train.text))
        search_and_display(E, train.text[qid], 7)
    end
end

<h1>Ejemplos de búsqueda (mensajes aleatorios sobre el train)</h1>

  0.000295 seconds (4 allocations: 96 bytes)


1,10394,0.0,PublimetroCol,Yaneth Waldman respondió a los rumores sobre presunta cirugía del hijo de Carolina Cruz https://t.co/FqHEakqzvt
2,22264,0.474,PublimetroCol,No ha podido recibir cirugía y alertaron a las autoridades https://t.co/8nDLnrDyeH
3,12134,0.481,PublimetroCol,"Tras varios meses, Lina Tejeiro revela la nueva cirugía que se hizo en el busto https://t.co/1UlF17Y7by"
4,9967,0.522,PublimetroChile,El hombre contó las diferentes cirugías que se realizó para parecerse al famoso cantante👇 https://t.co/02C9f1j5xg
5,9609,0.534,UniNoticias,"Caso Petito: autopsia de Brian Laundrie no deja resultados concluyentes y se harán más estudios, dice el abogado. https://t.co/sj3cmM1wgQ"
6,19661,0.539,AristeguiOnline,Yahir sale con éxito de operación de tumor https://t.co/zfrGIG5nH3 https://t.co/cV26z3S3sG
7,25293,0.542,SinEmbargoMX,"En el cuarto audio de la serie que la Gobernadora Layda Sansores ha difundido semanalmente en su programa, se escucha a Moreno Cárdenas dialogar en una consulta con un supuesto cosmetólogo o cirujano plástico, lo cual confirmó,su gusto por las frivolidades https://t.co/HXtY6Ffvfi https://t.co/TXdgZKcc3Y"


  0.000151 seconds (4 allocations: 96 bytes)


1,27988,0.0,epigmenioibarra,RT @nonalorehedzg03: @Drago237 @epigmenioibarra @rcanudasg @lizmorag @tinabrito1 @CompaPancho_ @HekGlezB @do_irma @haurrubi @mdgre2002 @yim…
2,28352,0.0,epigmenioibarra,RT @nonalorehedzg03: @Drago237 @epigmenioibarra @rcanudasg @lizmorag @tinabrito1 @CompaPancho_ @HekGlezB @do_irma @haurrubi @mdgre2002 @yim…
3,20296,0.024,epigmenioibarra,RT @LaraGue6: @Drago237 @epigmenioibarra @rcanudasg @lizmorag @tinabrito1 @CompaPancho_ @HekGlezB @do_irma @mdgre2002 @yimboroyimboro @more…
4,9844,0.035,epigmenioibarra,RT @nonalorehedzg03: @epigmenioibarra @rcanudasg @avril_prez @LalaL111444 @Lubruixa @luciataveras66 @lizmorag @mix_ivonne @edmeebmel1964 @m…
5,61,0.055,epigmenioibarra,RT @macamposr52: @epigmenioibarra @rcanudasg @avril_prez @nonalorehedzg03 @LalaL111444 @Lubruixa @luciataveras66 @lizmorag @mix_ivonne @edm…
6,7420,0.058,epigmenioibarra,RT @LupitaRivera453: @epigmenioibarra @rcanudasg @avril_prez @nonalorehedzg03 @Lubruixa @luciataveras66 @lizmorag @mix_ivonne @Angelyk4T @N…
7,20803,0.058,epigmenioibarra,RT @LupitaRivera453: @epigmenioibarra @rcanudasg @avril_prez @nonalorehedzg03 @Lubruixa @luciataveras66 @lizmorag @mix_ivonne @Angelyk4T @N…


  0.000311 seconds (4 allocations: 96 bytes)


1,14307,0.0,azucenau,"""Se detuvieron a tres personas, armamento y vehículos"": María Dolores del Río Sánchez, secretaria de seguridad pública de Sonora #AzucenaxFórmula"
2,26373,0.256,azucenau,"""El tema de la seguridad y el cerco en la zona dependerá de la seguridad de Kamala Harris"": secretario de Gobierno de la #CDMX, José Alfonso Suárez del Real y Aguilera en #AzucenaxFórmula"
3,17001,0.262,azucenau,"""El día de hoy se localizó la credencial perteneciente a #DebanhiEscobar"": Griselda Núñez, fiscal especializada en femincidios en #NuevoLeón #AzucenaxFórmula"
4,2742,0.28,azucenau,"""La secretaría de la mujer está trabajando junto a nosotros, queremos agradecer a los medios a visibilizar casos como el de Irma Ríos"": fiscal de #Oaxaca, Arturo Peimbert Calvo en #AzucenaxFórmula"
5,22053,0.28,azucenau,"""Son tres fosas las que componen una sola cisterna... ella ingresó y permaneció sola en ese lugar"": Luis Enrique Orozco, vicefiscal del MP de #NuevoLeón #DebanhiEscobar #AzucenaxFórmula"
6,532,0.298,abc_es,🔵 #DIRECTO Traspaso en el Gobierno: Iceta cede la cartera de Política Territorial a Isabel Rodríguez https://t.co/qy2qhpGckW https://t.co/f0XqTYFV8l
7,12746,0.304,AdriDelgadoRuiz,"“Ella -#ViridianaMoreno - monta su moto y monta al presunto asesino atrás” Luz María Rivera @DiccionarioALaD , Directora y columnista de @ElMercurioVer #ElDedoEnLaLlaga #HeraldoRadio📻🎙 @heraldodemexico 🗞📻📺📲 https://t.co/y2E6ei2UdJ"


In [45]:
display(@htl "<h1>Ejemplos de búsqueda (mensajes aleatorios sobre el test)</h1>")

for i in 1:3
    for qid in rand(1:length(test.text))
        search_and_display(E, test.text[qid], 7)
    end
end

<h1>Ejemplos de búsqueda (mensajes aleatorios sobre el test)</h1>

  0.000163 seconds (4 allocations: 96 bytes)


1,6728,0.164,RicardoAlemanMx,RT @JuanCarM17: @Pedro_V_H @RicardoAlemanMx @LAURAZAPATAM @kioscomayor @Jorge_Meade @irmisssas @ManuelH33813048 @JaimedelaGarzaM @AdrianaDa…
2,22788,0.165,RicardoAlemanMx,RT @venturaugo: @RicardoAlemanMx @lopezobrador_ @RicardoAnayaC ¡totalmente de acuerdo!
3,9170,0.166,RicardoAlemanMx,RT @RocioGarciaA1: @RicardoAlemanMx @XochitlGalvez @Claudiashein De acuerdo ☝🏼
4,5447,0.168,RicardoAlemanMx,RT @Pedro_V_H: @lopezobrador_ @zoerobledo @RicardoAlemanMx @YoSoyAlexLopez @Manzanares_E @juarezvero @manuelhidalgoh @EDELMIRO_GOMEZ @Faby_…
5,14708,0.169,epigmenioibarra,RT @ElenaSanchez_70: #PaseDeLista con @epigmenioibarra @Drago237 @alynmon @JoseL_Cortes @hekglez @hectorexafm @saltinmoryur @LaraGue6 @et…
6,15149,0.169,epigmenioibarra,RT @ElenaSanchez_70: #PaseDeLista con @epigmenioibarra @Drago237 @alynmon @JoseL_Cortes @hekglez @hectorexafm @saltinmoryur @LaraGue6 @et…
7,24735,0.169,epigmenioibarra,RT @ElenaSanchez_70: #PaseDeLista con @epigmenioibarra @Drago237 @alynmon @JoseL_Cortes @hekglez @hectorexafm @saltinmoryur @LaraGue6 @et…


  0.000141 seconds (4 allocations: 96 bytes)


1,7840,0.202,NTN24,"“Las regionales no tiene ninguna relación con sacar a Maduro del poder, es cambiar un gobernador por otro, pero eso no resuelve el problema de Venezuela”: @delsasolorzano sobre elecciones del 21 de noviembre https://t.co/H66sNEGxr1"
2,13690,0.206,NTN24,"RT @NTN24ve: #Atención Inicia jornada de diálogo en México, entre el régimen de Nicolás Maduro y la oposición venezolana. Según Jorge Rodrí…"
3,4541,0.225,teleSURtv,"#ENVIVO | El presidente de #Venezuela🇻🇪, @NicolasMaduro, brinda declaraciones en el marco del encuentro de activistas del Movimiento #SomosVenezuela 👉https://t.co/tqMKHfl0b4 https://t.co/ticuwZ8B9p"
4,5691,0.228,Radio_Formula,Los solicitantes ahora deberán juntar las firmas; Nicolás Maduro gobierna Venezuela desde el año 2013. https://t.co/NSUCdHzQ6s
5,18668,0.229,teleSURtv,#ENVIVO | El presidente de #Venezuela🇻🇪 @NicolasMaduro llama a los Gobernadores electos de la oposición a trabajar en conjunto Síguelo aquí➡️https://t.co/KoXryAkRTm https://t.co/ZQ4Q68R6Bu
6,20567,0.233,teleSURtv,"#ENVIVO | El presidente de #Venezuela🇻🇪 @NicolasMaduro expresa la resistencia que tuvo que pasar el país frente a los golpes contra la institucionalidad, la voluntad nacional y la capacidad política Síguelo aquí📲https://t.co/P9NavZC8Ln https://t.co/QPmn2yb4Ak"
7,9142,0.234,teleSURtv,"#ENVIVO | El presidente de #Venezuela🇻🇪 @NicolasMaduro expresa la resistencia que tuvo que pasar el país frente a los golpes contra la institucionalidad, la voluntad nacional y la capacidad política Síguelo aquí📲https://t.co/P9NavZC8Ln https://t.co/7jOL9FUHYB"


  0.000236 seconds (4 allocations: 96 bytes)


1,12101,0.05,abrahamendieta,RT @AristeguiOnline: Hubo 'operación de Estado' de Calderón para proteger a familia de Margarita Zavala en caso de guardería ABC: Zaldívar…
2,18588,0.156,abrahamendieta,"RT @AndreaChavezTre: Cuando se habló de cómo Felipe Calderón presionó a los ministros para proteger a la familia de Margarita Zavala, dueño…"
3,5877,0.18,abrahamendieta,"Se indigna Margarita Zavala cada vez en la Cámara de Diputados le recuerdan Guardería ABC. Hoy, Zaldivar cuenta como Calderón lo presionó con todo para intentar proteger a la familia Zavala. Prohibido olvidar."
4,19425,0.206,abrahamendieta,"No solo se buscaba proteger a Margarita Zavala, también a la familia Gómez del Campo, que no se nos olvide."
5,20337,0.241,julioastillero,"🚢 Participación de @xime_garmendia en #AstilleroInforma 👉🏼 En caso Mexicana de Aviación, Calderón benefició a su familia y Peña nadó de muertito: Garmendia 📺 Ve la entrevista completa en https://t.co/xbbhC0oJKa o en https://t.co/recOGrkFeJ https://t.co/BLpZWV824c"
6,10287,0.28,abrahamendieta,RT @catrina_nortena: #TRAIDORES_A_LA_PATRIA La sobrina plurinominal de la Peor Tragedia para México: Margarita Zavala y Felipe Calderón ht…
7,19250,0.302,abc_es,🔴 #ÚLTIMAHORA Destapado el acuerdo secreto de Zapatero con el que Sánchez oculta sus manejos en el caso Gali https://t.co/tz6XP2lplq


# Actividades
- Reproduzca el ejercicio de este notebook
- ¿Qué piensa de las diferencias de tamaño entre los documentos y las consultas? esto como afecta a la representación semántica.
- Implemente su búsqueda semántica. Si usa Julia considere `SimilaritySearch.jl` y si usa Python considere `faiss`.
- Utilece un extracto de 30 mensajes del test y compare el SBERT (usado en este notebook) contra  el modelo BERT `https://huggingface.co/bert-base-multilingual-cased` y BETO `https://huggingface.co/dccuchile/bert-base-spanish-wwm-cased`; adicionalmente incluya a los modelos tradicionales basados en vocabularios e índices invertidos.
- Reporte su notebook y anote sus soluciones a las preguntas planteadas. Reporte los resultados de sus implementaciones, compare contra las alternativas solicitadas. Discuta sus resultados. Finalice el reporte con reflexiones sobre el uso de nubes de puntos en lugar de bolsas de palabras tradicionales. Anoté sus conclusiones.

# Bibliografía
- [KSKW2015] Kusner, M., Sun, Y., Kolkin, N., & Weinberger, K. (2015, June). From word embeddings to document distances. In International conference on machine learning (pp. 957-966). PMLR.
- [PW2009] Pele, O., & Werman, M. (2009, September). Fast and robust earth mover's distances. In 2009 IEEE 12th international conference on computer vision (pp. 460-467). IEEE.
- [VSPU2017] Vaswani, A., Shazeer, N., Parmar, N., Uszkoreit, J., Jones, L., Gomez, A. N., ... & Polosukhin, I. (2017). Attention is all you need. Advances in neural information processing systems, 30.
- [RG2020] Reimers, N., & Gurevych, I. (2020). Making monolingual sentence embeddings multilingual using knowledge distillation. arXiv preprint arXiv:2004.09813.
- [CCFHKP2023] Cañete, J., Chaperon, G., Fuentes, R., Ho, J. H., Kang, H., & Pérez, J. (2023). Spanish pre-trained bert model and evaluation data. arXiv preprint arXiv:2308.02976.
